In [1]:
import numpy as np
from sklearn.datasets import make_blobs
import plotly.express as px

import logging
import sys

logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout)
    ]
)

COLORS = [
    "#f7dc6f",
    "#82e0aa",
    "#f1948a",
    "#499cef",
    "#f5b041",
    "#a569bd",
    "#e74c3c",
    "#2ecc71",
    "#3498db",
    "#e67e22",
    "#9b59b6",
    "#1abc9c",
    "#34495e",
    "#d35400",
    "#c0392b",
    "#16a085",
    "#2980b9",
    "#8e44ad",
]

def twospirals(n_points, noise=.5):
    n = np.sqrt(np.random.rand(n_points,1)) * 780 * (2*np.pi)/360
    d1x = -np.cos(n)*n + np.random.rand(n_points,1) * noise
    d1y = np.sin(n)*n + np.random.rand(n_points,1) * noise
    return (np.vstack((np.hstack((d1x,d1y)),np.hstack((-d1x,-d1y)))), np.hstack((np.zeros(n_points, dtype=int),np.ones(n_points, dtype=int))))

def plot_dataframe(df, title='', x='x', y='y', label='label'):
    df[label] = df[label].map(lambda x: str(x))
    fig = px.scatter(
        df.sort_values(by=label), 
        x=x, 
        y=y,
        color=label,
        color_discrete_map={str(i): COLORS[i] for i in range(len(COLORS))},
        title=title,
    )
    fig.update_traces(marker=dict(size=5, symbol='circle', opacity = 0.6))

    return fig

def plot_centroid(fig, centroid, colors = ['#a569bd'], marker_size=10, symbol = 'star', name = 'Centroid'):
    fig.add_scatter(
        x=[centroid[0]], 
        y=[centroid[1]], 
        mode='markers',
        marker=dict(
            size = marker_size,
            color = colors,
            symbol = symbol,
        ), 
        name=name
    )
    return fig

def create_constraints(labels, probability=0.01, seed=0):
    n_points = len(labels)
    constraints = np.zeros((n_points, n_points), dtype=int)
    state = np.random.RandomState(seed=seed)
    for i in range(n_points):
        for j in range(i +1, n_points):
            if state.rand() < probability:
                if labels[i] == labels[j]:
                    constraints[i, j] = 1
                    constraints[j, i] = 1
                elif labels[i] != labels[j]:
                    constraints[i, j] = -1
                    constraints[j, i] = -1
    return constraints

def draw_cl_constraints(fig, constraints, points):
    n_points = len(points)
    for i in range(n_points):
        for j in range(i + 1, n_points):
            if constraints[i, j] < 0:
                fig.add_scatter(
                    x=[points[i, 0], points[j, 0]], 
                    y=[points[i, 1], points[j, 1]], 
                    mode='lines',
                    line=dict(color='red', width=0.5, dash='dash'),
                )
    return fig

def draw_ml_constraints(fig, constraints, points):
    n_points = len(points)
    for i in range(n_points):
        for j in range(i + 1, n_points):
            if constraints[i, j] > 0:
                fig.add_scatter(
                    x=[points[i, 0], points[j, 0]], 
                    y=[points[i, 1], points[j, 1]], 
                    mode='lines',
                    line=dict(color='green', width=0.5),
                )
    return fig

In [2]:
X, y = make_blobs(n_samples=300, centers=10, cluster_std=1.0, random_state=42)
constraints = create_constraints(y, probability=0.01, seed=42)

In [5]:
from clustlib.fuzzy.lcvqe import LCVQE

lcvqe = LCVQE(
    n_clusters=10,
    constraints=constraints,
    max_iter=100,
    tol=1e-4
)
lcvqe.fit(X)

2025-07-10 09:54:57,119 - DEBUG - Fitting LCVQE model
2025-07-10 09:54:57,121 - DEBUG - Delta is None, convergence cannot be checked.
2025-07-10 09:54:57,122 - DEBUG - Iteration 0: Checking constraints
2025-07-10 09:54:57,150 - DEBUG - Iteration 0: Updating centroids
2025-07-10 09:54:57,166 - DEBUG - Iteration 0: Updating labels
2025-07-10 09:54:57,167 - DEBUG - Delta is None, convergence cannot be checked.
2025-07-10 09:54:57,167 - DEBUG - Iteration 1: Checking constraints
2025-07-10 09:54:57,181 - DEBUG - Iteration 1: Updating centroids
2025-07-10 09:54:57,182 - DEBUG - Iteration 1: Updating labels
2025-07-10 09:54:57,183 - DEBUG - Delta is None, convergence cannot be checked.
2025-07-10 09:54:57,183 - DEBUG - Iteration 2: Checking constraints
2025-07-10 09:54:57,193 - DEBUG - Iteration 2: Updating centroids
2025-07-10 09:54:57,194 - DEBUG - Iteration 2: Updating labels
2025-07-10 09:54:57,194 - DEBUG - Delta is None, convergence cannot be checked.
2025-07-10 09:54:57,195 - DEBUG - I

array([[-1.8069941 ,  7.68326852],
       [-1.7700408 ,  8.91611213],
       [-1.97132925,  8.52367237],
       [-1.78118179,  8.74954314],
       [-1.79131443,  8.81295896],
       [-1.86257907,  8.88580563],
       [-1.81130505,  8.66424649],
       [-1.80950166,  8.92751759],
       [-1.82984396,  8.82734938],
       [-1.79108163,  8.8324335 ]])

In [4]:
mtx = np.random.randint(-1, 2, size = (10, 10))
mtx = mtx - np.diag(np.diag(mtx))  # Remove diagonal elements
for i, j in np.argwhere(mtx > 0):
    print(f"{i} --- {j}")

0 --- 1
0 --- 2
0 --- 3
0 --- 5
0 --- 6
0 --- 8
1 --- 6
1 --- 9
2 --- 1
2 --- 3
2 --- 4
2 --- 8
3 --- 4
3 --- 5
3 --- 7
4 --- 3
4 --- 7
5 --- 7
6 --- 2
6 --- 3
6 --- 4
6 --- 5
6 --- 8
8 --- 0
8 --- 6
9 --- 0
9 --- 3


In [11]:
centroids = np.array([np.mean(X[y == label], axis = 0) for label in np.unique(y)])

In [12]:
centroids

array([[-2.74838844,  9.00706414],
       [ 4.57178916,  1.95986261],
       [-6.93249308, -6.69717547],
       [-8.83786767,  7.54111511],
       [ 2.33040889,  3.9558622 ],
       [-9.42239376,  9.35999659],
       [ 6.63614325, -5.55485231],
       [-6.382179  , -6.50569821],
       [-4.13139815,  0.20678414],
       [-1.24251742, -4.14405026]])